## How-to guide for Demand Forecasting use-case on Abacus.AI platform
This notebook provides you with a hands on environment to build a forecasting model using the Abacus.AI Python Client Library.

We'll be using the [Store Sales Data](https://s3.amazonaws.com/realityengines.exampledatasets/sales_forecasting/store_sales_w_holidays_timeseries.csv) and [Store Attributes](https://s3.amazonaws.com/realityengines.exampledatasets/sales_forecasting/store_metadata.csv) datasets, which contain information about multiple stores' sales and their attributes.

1. Install the Abacus.AI library.

In [1]:
!pip install abacusai

We'll also import pandas and pprint tools for neat visualization in this notebook.

In [2]:
import pandas as pd # A tool we'll use to download and preview CSV files
import pprint # A tool to pretty print dictionary outputs
pp = pprint.PrettyPrinter(indent=2)

2. Add your Abacus.AI [API Key](https://abacus.ai/app/profile/apikey) generated using the API dashboard as follows:

In [3]:
#@title Abacus.AI API Key

api_key = ''  #@param {type: "string"}

3. Import the Abacus.AI library and instantiate a client.

In [4]:
from abacusai import ApiClient
client = ApiClient(api_key)

## 1. Create a Project

Abacus.AI projects are containers that have datasets and trained models. By specifying a business **Use Case**, Abacus.AI tailors the deep learning algorithms to produce the best performing model catered specifically for your data.

We'll call the `list_use_cases` method to retrieve a list of the Use Cases currently available on the Abacus.AI platform.

In [5]:
client.list_use_cases()

[UseCase(use_case='UCPLUGANDPLAY',
   pretty_name='Plug & Play Your Tensorflow Model',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='EMBEDDINGS_ONLY',
   pretty_name='Vector Matching Engine',
   description='Upload embeddings and leverage our similarity search infrastructure.. Scale to high traffic, update your index in near realtime'),
 UseCase(use_case='MODEL_WITH_EMBEDDINGS',
   pretty_name='Tensorflow Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. Host your models on our infrastructure and get a JSON api with auto scaling and more!'),
 UseCase(use_case='TORCH_MODEL_WITH_EMBEDDINGS',
   pretty_name='PyTorch Model With Vector Matching Engine',
   description='Upload your already trained model and leverage our model serving infrastructure.. H

In this notebook, we're going to create a forecasting model using the Store Sales Data and Store Attributes datasets. The 'RETAIL' use case is best tailored for this situation.

In [6]:
#@title Abacus.AI Use Case

use_case = 'RETAIL'  #@param {type: "string"}

By calling the `describe_use_case_requirements` method we can view what datasets are required for this use_case.

In [7]:
for requirement in client.describe_use_case_requirements(use_case):
  pp.pprint(requirement.to_dict())

{ 'allowed_feature_mappings': { 'DATE': { 'allowed_feature_types': [ 'TIMESTAMP'],
                                          'description': 'Date (day, year '
                                                         'and/or month) that '
                                                         'corresponds to the '
                                                         'demand value.',
                                          'required': True},
                                'DEMAND': { 'allowed_feature_types': [ 'NUMERICAL'],
                                            'description': 'The demand value '
                                                           'you are '
                                                           'forecasting. (e.g '
                                                           'sales).',
                                            'required': True},
                                'FUTURE': { 'description': 'Known values ahead '
                    

Finally, let's create the project.

In [8]:
demand_forecasting_project = client.create_project(name='Store Sales Forecasting', use_case=use_case)
demand_forecasting_project.to_dict()

{'project_id': '1479d16004',
 'name': 'Store Sales Forecasting',
 'use_case': 'RETAIL',
 'created_at': '2021-11-23T19:13:03+00:00',
 'feature_groups_enabled': True}

**Note: When feature_groups_enabled is True then the use case supports feature groups (collection of ML features). Feature groups are created at the organization level and can be tied to a project to further use it for training ML models**

## 2. Add Datasets to your Project

Abacus.AI can read datasets directly from `AWS S3` or `Google Cloud Storage` buckets, otherwise you can also directly upload and store your datasets with Abacus.AI. For this notebook, we will have Abacus.AI read the datasets directly from a public S3 bucket's location.

We are using two datasets for this notebook. We'll tell Abacus.AI how each dataset should be used when creating it by tagging the dataset with a special Abacus.AI **Dataset Type**.
- [Store Sales Data](https://s3.amazonaws.com/realityengines.exampledatasets/sales_forecasting/store_sales_w_holidays_timeseries.csv) (**TIMESERIES**): 
This dataset contains information about store sales.
- [Sales Attributes](https://s3.amazonaws.com/realityengines.exampledatasets/sales_forecasting/store_metadata.csv) (**ITEM_ATTRIBUTES**): This dataset contains store attributes.

### Add the dataset to Abacus.AI

First we'll use Pandas to preview the file, then add it to Abacus.AI.

In [9]:
pd.read_csv('https://s3.amazonaws.com/realityengines.exampledatasets/sales_forecasting/store_sales_w_holidays_timeseries.csv')

/home/saim/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [10]:
pd.read_csv('https://s3.amazonaws.com/realityengines.exampledatasets/sales_forecasting/store_metadata.csv')

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


Using the Create Dataset API, we can tell Abacus.AI the public S3 URI of where to find the datasets. We will also give each dataset a Refresh Schedule, which tells Abacus.AI when it should refresh the dataset (take an updated/latest copy of the dataset).

If you're unfamiliar with Cron Syntax, Crontab Guru can help translate the syntax back into natural language: [https://crontab.guru/#0_12_\*_\*_\*](https://crontab.guru/#0_12_*_*_*)

**Note: This cron string will be evaluated in UTC time zone**

In [12]:
store_sales_dataset = client.create_dataset_from_file_connector(name='Store Sales Data1',
                                     table_name='Store_Sales_Data1',
                                     location='s3://realityengines.exampledatasets/sales_forecasting/store_sales_w_holidays_timeseries.csv',
                                     refresh_schedule='0 12 * * *')

store_attributes_dataset = client.create_dataset_from_file_connector(name='Store Attributes1', table_name='Store_Attributes1' ,
                                     location='s3://realityengines.exampledatasets/sales_forecasting/store_metadata.csv',
                                     refresh_schedule='0 12 * * *')

datasets = [store_sales_dataset, store_attributes_dataset]

## 3. Create Feature Groups and add them to your Project

Datasets are created at the organization level and can be used to create feature groups as follows:

In [14]:
feature_group = client.create_feature_group(table_name='Demand_forecasting',sql='SELECT * FROM Store_Sales_Data1')

Adding Feature Group to the project:

In [15]:
client.add_feature_group_to_project(feature_group_id=feature_group.feature_group_id,project_id = demand_forecasting_project.project_id)

Setting the Feature Group type according to the use case requirements:

In [16]:
client.set_feature_group_type(feature_group_id=feature_group.feature_group_id, project_id = demand_forecasting_project.project_id, feature_group_type= "TIMESERIES")

Check current Feature Group schema:

In [17]:
client.get_feature_group_schema(feature_group_id=feature_group.feature_group_id)

[Feature(name='Store',
   select_clause=None,
   feature_mapping=None,
   source_table='Store_Sales_Data1',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='DayOfWeek',
   select_clause=None,
   feature_mapping=None,
   source_table='Store_Sales_Data1',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='Date',
   select_clause=None,
   feature_mapping=None,
   source_table='Store_Sales_Data1',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='TIMESTAMP',
   data_type='DATETIME',
   columns=None,
   point_in_time_info=None),
 Feature(name='Sales',
   select_clause=None,
   feature_mapping=None,
   source_table='Store_Sales_Data1'

#### For each **Use Case**, there are special **Column Mappings** that must be applied to a column to fulfill use case requirements. We can find the list of available **Column Mappings** by calling the *Describe Use Case Requirements* API:

In [19]:
client.describe_use_case_requirements(use_case)[0].allowed_feature_mappings

{'ITEM_ID': {'description': 'The unique identifier of the item whose demand you are forecasting (e.g product Id, sku id).',
  'allowed_feature_types': ['CATEGORICAL'],
  'required': True},
 'DEMAND': {'description': 'The demand value you are forecasting. (e.g sales).',
  'allowed_feature_types': ['NUMERICAL'],
  'required': True},
 'DATE': {'description': 'Date (day, year and/or month) that corresponds to the demand value.',
  'allowed_feature_types': ['TIMESTAMP'],
  'required': True},
 'FUTURE': {'description': 'Known values ahead of time (eg., State Holidays, National Holidays etc.) that can be easily included in the training dataset.',
  'multiple': True,
  'required': False},
 'IGNORE': {'description': 'Ignore this column in training',
  'multiple': True,
  'required': False}}

In [21]:
client.set_feature_mapping(project_id = demand_forecasting_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='Store',feature_mapping='ITEM_ID')
client.set_feature_mapping(project_id = demand_forecasting_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='Sales',feature_mapping='DEMAND')
client.set_feature_mapping(project_id = demand_forecasting_project.project_id,feature_group_id= feature_group.feature_group_id, feature_name='Date',feature_mapping='DATE')

[Feature(name='Store',
   select_clause=None,
   feature_mapping='ITEM_ID',
   source_table='Store_Sales_Data1',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='DayOfWeek',
   select_clause=None,
   feature_mapping=None,
   source_table='Store_Sales_Data1',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='CATEGORICAL',
   data_type='STRING',
   columns=None,
   point_in_time_info=None),
 Feature(name='Date',
   select_clause=None,
   feature_mapping='DATE',
   source_table='Store_Sales_Data1',
   original_name=None,
   using_clause=None,
   order_clause=None,
   where_clause=None,
   feature_type='TIMESTAMP',
   data_type='DATETIME',
   columns=None,
   point_in_time_info=None),
 Feature(name='Sales',
   select_clause=None,
   feature_mapping='DEMAND',
   source_table='Store_S

For each required Feature Group Type within the use case, you must assign the Feature group to be used for training the model:

In [ ]:
client.use_feature_group_for_training(project_id=demand_forecasting_project.project_id, feature_group_id=feature_group.feature_group_id)

Now that we've our feature groups assigned, we're almost ready to train a model!

To be sure that our project is ready to go, let's call project.validate to confirm that all the project requirements have been met:

In [ ]:
demand_forecasting_project.validate()

## 4. Train a Model

For each **Use Case**, Abacus.AI has a bunch of options for training. We can call the *Get Training Config Options* API to see the available options.

In [ ]:
demand_forecasting_project.get_training_config_options()

In this notebook, we'll just train with the default options, but definitely feel free to experiment, especially if you have familiarity with Machine Learning.

In [ ]:
demand_forecasting_model = demand_forecasting_project.train_model(training_config={})
demand_forecasting_model.to_dict()

After we start training the model, we can call this blocking call that routinely checks the status of the model until it is trained and evaluated.

In [ ]:
demand_forecasting_model.wait_for_evaluation()       

**Note that model training might take some minutes to some hours depending upon the size of datasets, complexity of the models being trained and a variety of other factors**

## **Checkpoint** [Optional]
As model training can take an hours to complete, your page could time out or you might end up hitting the refresh button, this section helps you restore your progress:

In [ ]:
!pip install abacusai
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(indent=2)
api_key = ''  #@param {type: "string"}
from abacusai import ApiClient
client = ApiClient(api_key)
demand_forecasting_project = next(project for project in client.list_projects() if project.name == 'Store Sales Forecasting')
demand_forecasting_model = demand_forecasting_project.list_models()[-1]
demand_forecasting_model.wait_for_evaluation()

## Evaluate your Model Metrics

After your model is done training you can inspect the model's quality by reviewing the model's metrics

In [ ]:
pp.pprint(demand_forecasting_model.get_metrics().to_dict())

To get a better understanding on what these metrics mean, visit our [documentation](https://abacus.ai/app/help/useCases/RETAIL/training) page.

## 5. Deploy Model

After the model has been trained, we need to deploy the model to be able to start making predictions. Deploying a model will reserve cloud resources to host the model for Realtime and/or batch predictions.

In [ ]:
demand_forecasting_deployment = client.create_deployment(name='Store Sales Deployment',description='Store Sales Deployment',model_id=demand_forecasting_model.model_id)
demand_forecasting_deployment.wait_for_deployment()

After the model is deployed, we need to create a deployment token for authenticating prediction requests. This token is only authorized to predict on deployments in this project, so it's safe to embed this token inside of a user-facing application or website.

In [ ]:
deployment_token = demand_forecasting_project.create_deployment_token().deployment_token
deployment_token

## 6. Predict


Now that you have an active deployment and a deployment token to authenticate requests, you can make the `get_forecast` API call below.

This command will return a forecast under each percentile for the specified store. With the dataset we've specified, the forecast is made based on a pattern in the number of sales for the specified store/entity.

In [ ]:
ApiClient().get_forecast(deployment_token=deployment_token, 
               deployment_id=demand_forecasting_deployment.deployment_id, 
               query_data={"Store":"1"})